In [143]:
import math
import random
import numpy as np
import pandas as pd
import yfinance as yf
import torch
import torch.nn as nn
from torch.distributions import Dirichlet
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import deque

Hyperparameters

In [144]:
T_WINDOW      = 10       # lookback window for LSTM
ASSET_COUNT   = 5
ACTION_DIM    = ASSET_COUNT + 1  # +cash
STATE_DIM     = ACTION_DIM * T_WINDOW + ASSET_COUNT * 2  # allocations time series + indicators
HIDDEN_SIZE   = 128
LSTM_LAYERS   = 1

LR_INIT       = 3e-4
WEIGHT_DECAY  = 1e-4
TOTAL_UPDATES = 300
BATCH_SIZE    = 64
PPO_EPOCHS    = 5
CLIP_EPS      = 0.2
GAMMA         = 0.99
LAMBDA_GAE    = 0.95
ENTROPY_COEF  = 0.01
VALUE_COEF    = 0.5
MAX_GRAD_NORM = 0.5

N_STEP        = 3
VAL_INTERVAL  = 20
MAX_PATIENCE  = 5

Data Loading & Normalization

In [145]:
# Define the 10 assets (tickers) for the portfolio
# tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "BA", "NFLX", "NVDA", "META", "SBUX"]

# tickers = ["AAPL", "MSFT", "GOOGL", "SBUX", "TSLA"]
tickers = ["GME", "AMC", "SPCE", "NVAX", "NOK"]
# tickers = ["GME", "AMC", "BB", "NVAX", "NOK"]
# tickers = ["GME", "AMC", "HMC", "NVAX", "NOK"]
# Date range for historical data
start_date = "2017-01-01"
end_date   = "2023-12-31"

# Try to load price data from a local CSV, otherwise download using yfinance
data_file = "prices.csv"
try:
    prices_df = pd.read_csv(data_file, index_col=0, parse_dates=True)
    print("Loaded data from", data_file)
except FileNotFoundError:
    print("Downloading price data for tickers:", tickers)
    df = yf.download(tickers, start=start_date, end=end_date, interval="1d")
    # Extract the 'Close' prices from the MultiIndex DataFrame
    prices_df = df.xs('Close', axis=1, level='Price')
    prices_df.dropna(inplace=True)
    prices_df.to_csv(data_file)
    print("Data downloaded and saved to", data_file)

# Split data into training (first 4 years) and testing (last year)
full_train_df = prices_df[prices_df.index < "2023-01-01"]
test_df  = prices_df[prices_df.index >= "2023-01-01"]
full_train_prices = full_train_df.values  # shape: [train_days, 5]
test_prices  = test_df.values   # shape: [test_days, 5]
num_assets = full_train_prices.shape[1]
print(f"Training days: {full_train_prices.shape[0]}, Testing days: {test_prices.shape[0]}")

# Further split full training into training and validation (80/20)
split_index = int(0.8 * full_train_prices.shape[0])
train_prices = full_train_prices[:split_index]
val_prices   = full_train_prices[split_index:]

Loaded data from prices.csv
Training days: 1323, Testing days: 250


### observation normalization

In [146]:

class ObsNormalizer:
    def __init__(self, size, eps=1e-4):
        self.eps = eps
        self.mean = np.zeros(size)
        self.var  = np.ones(size)
        self.count = eps

    def update(self, x):
        batch_mean = x.mean(0)
        batch_var  = x.var(0)
        batch_count = x.shape[0]
        # Welford update
        delta = batch_mean - self.mean
        tot_count = self.count + batch_count
        new_mean = self.mean + delta * batch_count / tot_count
        m_a = self.var * self.count
        m_b = batch_var * batch_count
        M2 = m_a + m_b + delta**2 * self.count * batch_count / tot_count
        new_var = M2 / tot_count
        self.mean, self.var, self.count = new_mean, new_var, tot_count

    def normalize(self, x):
        return (x - self.mean) / np.sqrt(self.var + self.eps)

obs_norm = ObsNormalizer(STATE_DIM)

Reward Shaper

In [147]:
class RewardShaper:
    def __init__(self, window=30):
        self.window = window
        self.rews = deque(maxlen=window)
        self.peak = 0.0

    def reset(self):
        self.rews.clear()
        self.peak = 0.0

    def shape(self, raw, portval, turnover):
        # rolling Sharpe
        self.rews.append(raw)
        r = np.array(self.rews)
        sharpe = r.mean() / (r.std()+1e-6)
        # drawdown
        self.peak = max(self.peak, portval)
        dd = (self.peak - portval) / (self.peak+1e-6)
        # volatility penalty
        vol_pen = r.std()
        # turnover penalty
        to_pen = turnover
        return sharpe - 0.5*dd - 0.1*vol_pen - 0.1*to_pen

shaper = RewardShaper(window=30)

def compute_n_step_log_return(price_array, t, n_step, weights):
    """
    Compute log return from day t to t+n_step using given portfolio weights.
    - price_array: prices matrix
    - t: current day index
    - n_step: horizon (if beyond data end, use last available day)
    - weights: allocation fractions for assets and cash (length N+1, sum=1)
    Returns: log(growth_factor)
    """
    last_idx = min(t + n_step, price_array.shape[0] - 1)
    growth_factor = 0.0
    # weights includes cash as last element
    for i in range(len(weights)):
        if i == num_assets:  # cash index
            ratio = 1.0
        else:
            # price ratio from t to last_idx
            ratio = price_array[last_idx, i] / (price_array[t, i] + 1e-6)
        growth_factor += weights[i] * ratio
    return math.log(growth_factor + 1e-6)

State Builder

In [148]:
def get_market_indicators(prices, t, window=10):
    feats = []
    for i in range(ASSET_COUNT):
        if t<window:
            feats+= [1.0,0.0]
        else:
            win = prices[t-window+1:t+1,i]
            ma = win.mean()
            feats += [prices[t,i]/(ma+1e-6), np.std(np.diff(win)/(win[:-1]+1e-6))]
    return np.array(feats, dtype=np.float32)

def get_state(prices, t, alloc_history):
    # alloc_history: deque of last T_WINDOW allocation vectors
    alloc_seq = np.array(alloc_history).flatten()
    ind = get_market_indicators(prices, t)
    return np.concatenate([alloc_seq, ind])

Actor-Critic Network

In [149]:
class PPOActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        # LSTM to encode past allocations
        self.lstm = nn.LSTM(ACTION_DIM, HIDDEN_SIZE, 
                            num_layers=LSTM_LAYERS, batch_first=True)
        self.ln1  = nn.LayerNorm(HIDDEN_SIZE)
        # MLP after concat
        self.fc1 = nn.Linear(HIDDEN_SIZE + 2*ASSET_COUNT, HIDDEN_SIZE)
        self.ln2 = nn.LayerNorm(HIDDEN_SIZE)
        # policy head: Dirichlet alphas
        self.policy = nn.Linear(HIDDEN_SIZE, action_dim)
        # value head
        self.value  = nn.Linear(HIDDEN_SIZE, 1)

        # orthogonal init
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight, gain=np.sqrt(2))
                nn.init.constant_(m.bias, 0.0)

    def forward(self, alloc_seq, ind_feats, hx=None):
        # alloc_seq: [B, T_WINDOW, ACTION_DIM]
        out, hx = self.lstm(alloc_seq, hx)  # out: [B, T_WINDOW, H]
        h = out[:,-1]                       # take last step
        h = self.ln1(h)
        x = torch.cat([h, ind_feats], dim=-1)
        x = F.relu(self.ln2(self.fc1(x)))
        alpha = F.softplus(self.policy(x)) + 1e-3
        v     = self.value(x)
        return alpha, v, hx

### PPO Agent

In [150]:
class PPOAgent:
    def __init__(self, state_dim, action_dim):
        self.net = PPOActorCritic(state_dim, action_dim).float()
        self.optim = optim.AdamW(self.net.parameters(), 
                                 lr=LR_INIT, weight_decay=WEIGHT_DECAY)
        self.scheduler = optim.lr_scheduler.CosineAnnealingLR(
            self.optim, TOTAL_UPDATES, eta_min=1e-6)
        self.buffer = []  # collect trajectories

    def select_action(self, state, hx=None):
        seq, ind = state
        seq_t = torch.from_numpy(seq[None]).float()
        ind_t = torch.from_numpy(ind[None]).float()
        alpha, v, hx = self.net(seq_t, ind_t, hx)
        dist = Dirichlet(alpha.squeeze(0))
        a = dist.sample()
        return a.detach().cpu().numpy(), dist.log_prob(a).item(), v.item(), hx

    def store(self, transition):
        self.buffer.append(transition)

    def update(self):
        # extract batch
        seqs, inds, acts, oldlps, rews, vals, dones = zip(*self.buffer)
        T = len(rews)
        # to tensors
        seq_t = torch.tensor(seqs, dtype=torch.float32)
        ind_t = torch.tensor(inds, dtype=torch.float32)
        act_t = torch.tensor(acts, dtype=torch.float32)
        lp_t  = torch.tensor(oldlps, dtype=torch.float32)
        rew   = np.array(rews, dtype=np.float32)
        val   = np.array(vals, dtype=np.float32)
        done  = np.array(dones, dtype=np.bool_)

        # compute GAE & returns
        adv = np.zeros(T, np.float32)
        ret = np.zeros(T, np.float32)
        next_val = 0.0
        gae = 0.0
        for i in reversed(range(T)):
            mask = 0.0 if done[i] else 1.0
            delta = rew[i] + GAMMA*next_val*mask - val[i]
            gae   = delta + GAMMA*LAMBDA_GAE*mask*gae
            adv[i] = gae
            next_val = val[i]
        ret = adv + val
        # normalize adv
        adv = (adv - adv.mean())/(adv.std()+1e-6)

        # convert
        adv_t = torch.tensor(adv, dtype=torch.float32)
        ret_t = torch.tensor(ret, dtype=torch.float32)

        # PPO updates
        for _ in range(PPO_EPOCHS):
            # single large batch (can minibatch if desired)
            alpha, v, _ = self.net(seq_t, ind_t)
            dist = Dirichlet(alpha)
            lp_new = dist.log_prob(act_t)
            ratio = torch.exp(lp_new - lp_t)
            s1 = ratio * adv_t
            s2 = torch.clamp(ratio, 1-CLIP_EPS, 1+CLIP_EPS) * adv_t
            loss_a = -torch.min(s1, s2).mean()
            loss_c = F.mse_loss(v.squeeze(1), ret_t)
            loss_e = -ENTROPY_COEF * dist.entropy().mean()
            loss = loss_a + VALUE_COEF*loss_c + loss_e

            self.optim.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(self.net.parameters(), MAX_GRAD_NORM)
            self.optim.step()
        self.scheduler.step()
        self.buffer.clear()

### Validation

In [151]:
def evaluate_policy_ppo(net, price_array, initial_alloc, T_WINDOW, ASSET_COUNT, N_STEP):
    """
    Deterministic evaluation of the trained PPO policy:
      - net: the PPOActorCritic network
      - price_array: np.array of shape [days, ASSET_COUNT]
      - initial_alloc: length-(ASSET_COUNT+1) starting allocation (fractions sum to 1)
      - T_WINDOW: how many past allocations the LSTM expects
      - N_STEP: unused here (we use 1-step returns for eval)
    Returns: list of daily portfolio values
    """
    net.eval()
    ACTION_DIM = ASSET_COUNT + 1
    days = price_array.shape[0]

    # initialize allocation history with the initial_alloc repeated
    alloc_hist = deque([initial_alloc.copy()] * T_WINDOW, maxlen=T_WINDOW)
    port_vals = [1.0]
    alloc = initial_alloc.copy()

    with torch.no_grad():
        for t in range(days - 1):
            # build the two inputs
            seq_np = np.stack(alloc_hist, axis=0)           # shape [T_WINDOW, ACTION_DIM]
            ind_np = get_market_indicators(price_array, t)  # shape [2*ASSET_COUNT]

            seq_t = torch.from_numpy(seq_np[None]).float()  # [1, T_WINDOW, ACTION_DIM]
            ind_t = torch.from_numpy(ind_np[None]).float()  # [1, 2*ASSET_COUNT]

            # forward pass
            alpha_t, _, _ = net(seq_t, ind_t)
            alpha = alpha_t.squeeze(0).cpu().numpy()        # [ACTION_DIM]

            # deterministic allocation = mean of Dirichlet = alpha / sum(alpha)
            if alpha.sum() <= 0:
                alloc = np.ones_like(alpha) / ACTION_DIM
            else:
                alloc = alpha / alpha.sum()

            # compute 1‑step portfolio growth
            growth = 0.0
            for i in range(ACTION_DIM):
                if i == ASSET_COUNT:  # cash
                    ratio = 1.0
                else:
                    ratio = price_array[t+1, i] / (price_array[t, i] + 1e-6)
                growth += alloc[i] * ratio

            port_vals.append(port_vals[-1] * growth)

            # update history
            alloc_hist.append(alloc)

    return port_vals

def evaluate_val_perf(net, price_array, initial_alloc):
    # run the deterministic eval
    vals = evaluate_policy_ppo(
        net,
        price_array,
        initial_alloc,
        T_WINDOW  = T_WINDOW,
        ASSET_COUNT = ASSET_COUNT,
        N_STEP = N_STEP
    )
    # return log of final portfolio value
    return math.log(vals[-1] + 1e-6)

### Training Loop with Early Stopping

In [152]:
def run_training():
    agent = PPOAgent(STATE_DIM, ACTION_DIM)
    best_val = -float('inf'); patience = 0
    initial_alloc = np.zeros(ACTION_DIM, np.float32); initial_alloc[-1] = 1.0

    for update in range(1, TOTAL_UPDATES+1):
        # collect one rollout
        shaper.reset()
        alloc_hist = deque([initial_alloc.copy() for _ in range(T_WINDOW)], maxlen=T_WINDOW)
        portval = 1.0
        hx = None

        for t in range(train_prices.shape[0]-N_STEP-1):
            state = (np.array(alloc_hist), 
                     get_market_indicators(train_prices, t))
            # normalize obs
            flat = state[0].flatten(); obs = np.concatenate([flat, state[1]])
            obs_norm.update(obs[None])
            norm_obs = (obs - obs_norm.mean)/np.sqrt(obs_norm.var+1e-4)
            # re-split
            seq = norm_obs[:ACTION_DIM*T_WINDOW].reshape(T_WINDOW, ACTION_DIM)
            ind = norm_obs[ACTION_DIM*T_WINDOW:]
            
            # action & logprob
            a, lp, v, hx = agent.select_action((seq, ind), hx)
            # compute turnover
            turnover = np.sum(np.abs(a - alloc_hist[-1]))
            alloc_hist.append(a)
            # reward
            raw = compute_n_step_log_return(train_prices, t, N_STEP, a)
            portval *= math.exp(raw)
            r = shaper.shape(raw, portval, turnover)
            done = False
            # store transition
            agent.store((seq, ind, a, lp, r, v, done))

        agent.update()

        # # validation & early stop
        # if update % VAL_INTERVAL == 0:
        #     # evaluate deterministically on val set...
        #     val_perf = evaluate_val_perf(
        #             agent.net,
        #             val_prices,      # your validation prices np.array
        #             initial_alloc       # same initial alloc used in training
        #         )
        #     print(f"Update {update}: val={val_perf:.4f}  best={best_val:.4f}")
        #     if val_perf > best_val:
        #         best_val = val_perf
        #         patience = 0
        #         torch.save(agent.net.state_dict(), 'best_ppo.pt')
        #         print("  ↳ new best! checkpointing model")
        #     else:
        #         patience += 1
        #         print(f"  ↳ no improvement ({patience}/{MAX_PATIENCE})")
        #         if patience >= MAX_PATIENCE:
        #             print("Early stopping.")
        #             break
        if update % 10 == 0:
            print(f"Update {update}:  portfolio value={portval:.4f}")
    # restore best
    agent.net.load_state_dict(torch.load('best_ppo.pt'))
    return agent.net

Evaluation

In [153]:
initial_alloc = np.zeros(ACTION_DIM, np.float32); initial_alloc[-1] = 1.0
models = []
for i in range(10):
    net = run_training()
    best_value = -float('inf')
    port_values = evaluate_policy_ppo(
        net,
        test_prices,
        initial_alloc,
        T_WINDOW=10,
        ASSET_COUNT=5,
        N_STEP=3
    )
    if best_value < port_values[-1]:
        best_value = port_values[-1]
        # best_model = model
    print("Test portfolio value:", port_values[-1])    
    models.append(net)

print("Test final return:", (best_value-1)*100, "%")

Update 10:  portfolio value=91.1247
Update 20:  portfolio value=1171.4208
Update 30:  portfolio value=183.7978
Update 40:  portfolio value=77.9434
Update 50:  portfolio value=415.2174
Update 60:  portfolio value=397.4070
Update 70:  portfolio value=348.6648
Update 80:  portfolio value=2401.6916
Update 90:  portfolio value=994.4806
Update 100:  portfolio value=635.4881
Update 110:  portfolio value=169.8166
Update 120:  portfolio value=199.0055
Update 130:  portfolio value=2174.3429
Update 140:  portfolio value=764.2259
Update 150:  portfolio value=4449.1161
Update 160:  portfolio value=2804.5081
Update 170:  portfolio value=225.4980
Update 180:  portfolio value=5038.8210
Update 190:  portfolio value=1225.9553
Update 200:  portfolio value=1151.2056
Update 210:  portfolio value=1488.9899
Update 220:  portfolio value=9772.9722
Update 230:  portfolio value=396.8083
Update 240:  portfolio value=3233.9323
Update 250:  portfolio value=378.7924
Update 260:  portfolio value=1630.1881
Update 270: